In [1]:
import pandas as pd
import numpy as np

In [171]:
df_gare = pd.read_csv('data/dvf-clean-work.csv')

C:\Users\JouJo\Anaconda3\envs\Python3_9\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Clean DF_Gare

In [172]:
df_gare = df_gare.iloc[:, 2:]

In [173]:
columns_to_keep = ['date_mutation', 'nature_mutation', 'type_de_voie', 'code_postal', 'code_departement', 'valeur_fonciere',
                   'section', 'nombre_de_lots', 'nombre_pieces_principales', 'commune',
                   'sncf_ngare1', 'sncf_ngare2', 'sncf_ngare3', 'sncf_min_distance',
                   'ratp_ngare1', 'ratp_ngare2', 'ratp_ngare3', 'ratp_min_distance',
                   'surface_batiment', 'surface', 'prix_mcarre', 'date', 'year']

In [174]:
df_gare = (df_gare
           .loc[:, columns_to_keep]
           .assign(gare_proche = lambda x: np.where(
               x.sncf_min_distance > x.ratp_min_distance, x.ratp_min_distance, x.sncf_min_distance))
           .assign(code_postal = lambda x: x.code_postal.astype('int').astype('str'))
           # Remove locations that are more than 10km away from any station (they have incorrect geocodes)
           .query("gare_proche < 10")
          )

In [175]:
df_gare.head(3)

,date_mutation,nature_mutation,type_de_voie,code_postal,code_departement,valeur_fonciere,section,nombre_de_lots,nombre_pieces_principales,commune,...,ratp_ngare1,ratp_ngare2,ratp_ngare3,ratp_min_distance,surface_batiment,surface,prix_mcarre,date,year,gare_proche
0,06/01/2016,Vente,RUE,92000,92,"215000,00",CS,1,3.0,NANTERRE,...,1.0,3.0,4.0,0.328910,119.0,74.0,1806.722689,2016-06-01,2016,0.328910
1,06/01/2016,Vente,RUE,92000,92,"215000,00",CS,2,2.0,NANTERRE,...,1.0,3.0,4.0,0.328910,119.0,45.0,1806.722689,2016-06-01,2016,0.328910
2,07/01/2016,Vente,RUE,92400,92,"320000,00",AY,2,3.0,COURBEVOIE,...,0.0,2.0,12.0,1.101011,78.0,78.0,4102.564103,2016-07-01,2016,0.225391


### Ecoles

In [176]:
df_ecoles = pd.read_csv('data/dataidf-ecoles.csv').iloc[:, 1:]

In [177]:
df_ecoles = (df_ecoles
 .pivot(index = 'code_postal',
              columns = 'type_etablissement',
              values = 'code_etablissement')
 .reset_index()
 .assign(code_postal = lambda x: x.code_postal.astype('str'))
 .loc[:, ['code_postal', 'COLLEGE', 'ELEMENTAIRE', 'LYCEE', 'MATERNELLE']])

In [178]:
df_ecoles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   code_postal  678 non-null    object 
 1   COLLEGE      473 non-null    float64
 2   ELEMENTAIRE  527 non-null    float64
 3   LYCEE        359 non-null    float64
 4   MATERNELLE   481 non-null    float64
dtypes: float64(4), object(1)
memory usage: 26.6+ KB


### Area verts

In [179]:
df_verts = pd.read_csv('data/dataidf-area-verts.csv').iloc[:, 1:].assign(code_postal=lambda x: x.code_postal.astype('str'))

In [180]:
pd.merge(df_ecoles, df_verts, left_on='code_postal', right_on='code_postal', how='outer')

,code_postal,COLLEGE,ELEMENTAIRE,LYCEE,MATERNELLE,n_area_verts,avg_area_ha
0,75001,1.0,4.0,1.0,2.0,4.0,15.160000
1,75002,2.0,6.0,4.0,2.0,NaN,NaN
2,75003,4.0,8.0,5.0,5.0,NaN,NaN
3,75004,5.0,12.0,5.0,3.0,2.0,1.185000
4,75005,8.0,14.0,8.0,11.0,1.0,20.620000
...,...,...,...,...,...,...,...
790,94077,NaN,NaN,NaN,NaN,12.0,4.912500
791,94078,NaN,NaN,NaN,NaN,9.0,24.470000
792,94079,NaN,NaN,NaN,NaN,6.0,1.490000
793,94080,NaN,NaN,NaN,NaN,3.0,1.836667


### Population

In [181]:
df_population = pd.read_csv('data/insee-population2015.csv').iloc[:, 1:].assign(code_postal=lambda x: x.code_postal.astype('int').astype('str'))

In [182]:
df_population.head()

,code_postal,nom_com,population
0,9400,SAURAT,668
1,9200,MONTJOIE-EN-COUSERANS,1053
2,9460,ROUZE,90
3,9270,MAZERES,3778
4,9240,DURBAN-SUR-ARIZE,178


In [183]:
df_population = (df_population
 .query("code_postal.str.len() > 4")
 .groupby('code_postal', as_index=False)
 .agg(population=('population', 'sum'))
)
df_population.head(3)

,code_postal,population
0,75001,17100
1,75002,22390
2,75003,35991


### Revenu

In [184]:
df_revenu = pd.read_csv('data/iris-revenus2018.csv').iloc[:, 1:].assign(code_postal=lambda x: x.code_postal.astype('str'))

In [185]:
df_revenu

,code_postal,revenu_moyen
0,75001,35295.714286
1,75002,32617.000000
2,75003,33231.764706
3,75004,32447.500000
4,75005,34903.200000
...,...,...
124,94077,21253.750000
125,94078,15877.692308
126,94079,23703.333333
127,94080,33201.176471


In [186]:
df_final = pd.merge(df_gare, df_population, left_on='code_postal', right_on='code_postal', how='left')

In [187]:
df_final = pd.merge(df_final, df_ecoles, left_on='code_postal', right_on='code_postal', how='left')
df_final = pd.merge(df_final, df_verts, left_on='code_postal', right_on='code_postal', how='left')

In [190]:
df_final = pd.merge(df_final, df_revenu, left_on='code_postal', right_on='code_postal', how='left')

In [192]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 462592 entries, 0 to 462591
Data columns (total 32 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   date_mutation              462592 non-null  object 
 1   nature_mutation            462592 non-null  object 
 2   type_de_voie               462592 non-null  object 
 3   code_postal                462592 non-null  object 
 4   code_departement           462592 non-null  int64  
 5   valeur_fonciere            462592 non-null  object 
 6   section                    462592 non-null  object 
 7   nombre_de_lots             462592 non-null  int64  
 8   nombre_pieces_principales  462592 non-null  float64
 9   commune                    462592 non-null  object 
 10  sncf_ngare1                462592 non-null  float64
 11  sncf_ngare2                462592 non-null  float64
 12  sncf_ngare3                462592 non-null  float64
 13  sncf_min_distance          46